In [1]:
import pandas as pd
import numpy as np
import re

## Movie Lens original data

In [2]:
DATA_PATH="ml-1m/ml-1m/"
df_movies = pd.read_csv(DATA_PATH+"movies.dat", sep="::", header=None)
df_movies.columns = ["ItemID","Title","Genres"]
df_movies["Year"] = df_movies.Title.apply(lambda x: re.match(".*\((\w+)\)", x)[1])
df_movies.head()

C:\Users\kdhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,ItemID,Title,Genres,Year
0,1,Toy Story (1995),Animation|Children's|Comedy,1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [3]:
map_occ = {0:  "other",
1:  "academic/educator",
2:  "artist",
3:  "clerical/admin",
4:  "college/grad student",
5:  "customer service",
6:  "doctor/health care",
7:  "executive/managerial",
8:  "farmer",
9:  "homemaker",
10:  "K-12 student",
11:  "lawyer",
12:  "programmer",
13:  "retired",
14:  "sales/marketing",
15:  "scientist",
16:  "self-employed",
17:  "technician/engineer",
18:  "tradesman/craftsman",
19:  "unemployed",
20:  "writer"}



df_users = pd.read_csv(DATA_PATH+"users.dat", sep="::", header=None)
df_users.columns = ["UserID","Gender","Age","Occupation","Zip-code"]
#df["Year"] = df.Title.applylambda x: re.match".*\\w+)\)", x)[1])
df_users["Occupation2"] = df_users.Occupation.apply(lambda x: map_occ[x])
df_users.head(10)

C:\Users\kdhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,UserID,Gender,Age,Occupation,Zip-code,Occupation2
0,1,F,1,10,48067,K-12 student
1,2,M,56,16,70072,self-employed
2,3,M,25,15,55117,scientist
3,4,M,45,7,02460,executive/managerial
4,5,M,25,20,55455,writer
5,6,F,50,9,55117,homemaker
6,7,M,35,1,06810,academic/educator
7,8,M,25,12,11413,programmer
8,9,M,25,17,61614,technician/engineer
9,10,F,35,1,95370,academic/educator


In [24]:
df_ratings = pd.read_csv(DATA_PATH+"/ratings.dat", sep="::", header=None)
df_ratings.columns = ["UserID","ItemID","Rating","Timestamp"]
df_ratings["time"] = df_ratings.Timestamp.apply(lambda x: pd.datetime.fromtimestamp(x))


C:\Users\kdhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [25]:
df_ratings.head()

,UserID,ItemID,Rating,Timestamp,time
0,1,1193,5,978300760,2000-12-31 14:12:40
1,1,661,3,978302109,2000-12-31 14:35:09
2,1,914,3,978301968,2000-12-31 14:32:48
3,1,3408,4,978300275,2000-12-31 14:04:35
4,1,2355,5,978824291,2001-01-06 15:38:11


## NCF Paper data

In [18]:
NCF_DATA_PATH="ncf/Data/"
df = pd.read_csv(NCF_DATA_PATH+"ml-1m.train.rating", sep="\t", header=None)
df.columns = ["UserID","ItemID","Rating","Timestamp"]
len(df.UserID.unique()), len(df.ItemID.unique())
train_items = set(df.ItemID.values)

In [19]:
df = pd.read_csv(NCF_DATA_PATH+"ml-1m.test.rating", sep="\t", header=None)
df.columns = ["UserID","ItemID","Rating","Timestamp"]
len(df.UserID.unique()), len(df.ItemID.unique())
test_items = set(df.ItemID.values)

In [20]:
test_items-train_items

{1805, 3569}

## Movie Lens Data - split train test data

In [4]:
# generate test data - ratings
df_ratings = pd.read_csv(DATA_PATH+"ratings.dat", sep="::", header=None)
df_ratings.columns = ["UserID","ItemID","Rating","Timestamp"]
#df_ratings["time"] = df_ratings.Timestamp.apply(lambda x: pd.datetime.fromtimestamp(x))

C:\Users\kdhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


df1 = df_ratings.groupby("UserID").count()
users = set(df1[df1["ItemID"]>=300].index)
df_ratings["condition"] = df_ratings["UserID"].apply(lambda x: True if x in users else False)
df_ratings[df_ratings["condition"]]

df_ratings = df_ratings.sort_values(["UserID","ItemID"])
df_ratings.head()

In [5]:
_id=0
id_map = dict()

def mapit(i):
    global _id
    if not i in id_map:
        id_map[i] = _id
        _id += 1
        
    return id_map[i]
df_ratings = df_ratings.sort_values("UserID")
df_ratings["UserID"] = df_ratings["UserID"].apply(lambda x: mapit(x))
_id=0
id_map = dict()
df_ratings = df_ratings.sort_values("ItemID")
df_ratings["ItemID"] = df_ratings["ItemID"].apply(lambda x: mapit(x))
df_ratings = df_ratings.sort_values(["UserID","ItemID"])

In [6]:
df_ratings.head()

,UserID,ItemID,Rating,Timestamp
40,0,0,5,978824268
25,0,47,5,978824351
39,0,144,5,978301777
44,0,253,4,978300760
23,0,513,5,978824195


df_ratings.sort_values(["UserID","ItemID"])#.sort_values("Timestamp", ascending=False)

In [7]:
df_ratings = df_ratings.sort_values("Timestamp", ascending=False)
df_ratings.head()

,UserID,ItemID,Rating,Timestamp
825603,4957,1744,4,1046454590
825724,4957,3040,4,1046454548
825731,4957,2432,3,1046454548
825438,4957,1305,5,1046454443
825793,4957,2206,1,1046454338


In [22]:
from collections import defaultdict

count_per_user = 1
count_per_user_valid = 0
user_counts = defaultdict(int)
rows = []
rows_valid = []
for index, row in df_ratings.iterrows():
    if user_counts[row["UserID"]] < count_per_user:
        rows += [index]
        user_counts[row["UserID"]] +=1
    elif user_counts[row["UserID"]] < count_per_user+count_per_user_valid:
        rows_valid += [index]
        #user_counts[row["UserID"]] +=1

In [23]:
len(rows),len(rows_valid)

(6040, 0)

In [24]:
df_test_ratings = df_ratings.loc[rows]
df_valid_ratings = df_ratings.loc[rows_valid]
df_train_ratings = df_ratings.loc[list(set(df_ratings.index)-set(df_test_ratings.index)-set(df_valid_ratings.index))]

In [25]:
df_test_ratings = df_ratings.loc[rows]
df_train_ratings = df_ratings.loc[list(set(df_ratings.index)-set(df_test_ratings.index))]

In [26]:
df_test_ratings = df_test_ratings.sort_values(["UserID","ItemID"])
df_test_ratings.head()

,UserID,ItemID,Rating,Timestamp
25,0,47,5,978824351
136,1,1737,3,978300174
232,2,1900,4,978298504
235,3,2743,4,978294282
258,4,279,2,978246585


In [27]:
df_valid_ratings = df_valid_ratings.sort_values(["UserID","ItemID"])
df_valid_ratings.head()

,UserID,ItemID,Rating,Timestamp


In [28]:
df_train_ratings = df_train_ratings.sort_values(["UserID","ItemID"])
df_train_ratings.head()

,UserID,ItemID,Rating,Timestamp
40,0,0,5,978824268
39,0,144,5,978301777
44,0,253,4,978300760
23,0,513,5,978824195
49,0,517,4,978302149


In [34]:
df_train_ratings.ItemID.max()

3705

df_test_ratings.to_csv("data/movielens.test.ratings", index=False)
df_train_ratings.to_csv("data/movielens.train.ratings", index=False)

### Implict Feedback and Negative Sampling

In [37]:
user_item_pairs = defaultdict(set)

for x in df_ratings[["UserID", "ItemID"]].values:
    # add all +ve to set
    user_item_pairs[x[0]].add(x[1])

num_items = df_ratings["ItemID"].max()

def get_negs(u):
    negs = []
    for _i in range(num_negs_per_positive):
        # generate negatives and add to dict
        j = np.random.randint(num_items)
        while (u,j) in user_item_pairs[u]:
            j = np.random.randint(num_items)
        negs += [j]
        user_item_pairs[u].add(j)
    return negs

num_negs_per_positive = 99
df_test_ratings["Negatives"] = df_test_ratings["UserID"].apply(lambda x: get_negs(x))
#num_negs_per_positive = 99
#df_valid_ratings["Negatives"] = df_valid_ratings["UserID"].apply(lambda x: get_negs(x))
num_negs_per_positive = 7
df_train_ratings["Negatives"] = df_train_ratings["UserID"].apply(lambda x: get_negs(x))

In [38]:
df_test_ratings.head()

,UserID,ItemID,Rating,Timestamp,Negatives
25,0,47,5,978824351,"[3515, 3231, 1032, 1865, 1165, 2577, 2461, 279..."
136,1,1737,3,978300174,"[2577, 1010, 2304, 564, 73, 3636, 3298, 1874, ..."
232,2,1900,4,978298504,"[850, 933, 3324, 2238, 2205, 2142, 866, 158, 1..."
235,3,2743,4,978294282,"[1266, 752, 2043, 313, 3609, 1693, 2168, 3144,..."
258,4,279,2,978246585,"[954, 1849, 1503, 2459, 1170, 3392, 558, 2274,..."


In [39]:
df_valid_ratings.head()

,UserID,ItemID,Rating,Timestamp


In [40]:
df_train_ratings = df_train_ratings.sort_values(["UserID","ItemID"])
df_train_ratings.head()

,UserID,ItemID,Rating,Timestamp,Negatives
40,0,0,5,978824268,"[1042, 578, 1916, 928, 1237, 403, 907]"
39,0,144,5,978301777,"[2223, 784, 1687, 2319, 2201, 969, 2433]"
44,0,253,4,978300760,"[3443, 1650, 948, 2722, 644, 1002, 688]"
23,0,513,5,978824195,"[432, 3374, 2218, 3452, 247, 685, 1739]"
49,0,517,4,978302149,"[3010, 539, 695, 2646, 1686, 3061, 2104]"


In [33]:
len(df_train_ratings.ItemID.unique())

3704

In [41]:
df_test_ratings.to_pickle("data/movielens.test.data.wo_repl")
#df_valid_ratings.to_pickle("data/movielens.valid.data.1")
df_train_ratings.to_pickle("data/movielens.train.data.wo_repl")

%%time
pd.read_pickle("data/movielens.train.data")

In [36]:
!ls -h data/

movielens.test.data
movielens.test.data.1
movielens.test.data.original
movielens.train.data
movielens.train.data.1
movielens.train.data.original
movielens.valid.data


In [38]:
len(df_train_ratings), len(df_test_ratings)

(994169, 6040)

In [20]:
df1 = pd.read_pickle("data/movielens.test.data.original")
df1.head()

,UserID,ItemID,Negatives,Rating
0,0,25,"[1064, 174, 2791, 3373, 269, 2678, 1902, 3641,...",1
1,1,133,"[1072, 3154, 3368, 3644, 549, 1810, 937, 1514,...",1
2,2,207,"[2216, 209, 2347, 3, 1652, 3397, 383, 2905, 22...",1
3,3,208,"[3023, 1489, 1916, 1706, 1221, 1191, 2671, 81,...",1
4,4,222,"[1794, 3535, 108, 593, 466, 2048, 854, 1378, 1...",1


In [153]:
df1 = pd.read_pickle("data/movielens.train.data.original")
df1 = df1.sort_values(["UserID","ItemID"])
df1.shape

(994169, 4)

In [115]:
df11 = pd.read_pickle("data/movielens.test.data.original")
df11 = df11.sort_values(["UserID","ItemID"])
df11.shape

(6040, 4)

In [131]:
df13 = pd.concat([df1, df11.drop("Negatives", axis=1)])

In [147]:
df13.sort_values(["UserID","Timestamp"]).head()

,ItemID,Rating,Timestamp,UserID
51,31,4,978300019.0,0
48,22,5,978300055.0,0
50,27,4,978300055.0,0
49,37,5,978300055.0,0
47,24,3,978300103.0,0


In [143]:
df13.groupby("UserID").count().sort_values("ItemID", ascending=True).head()

,ItemID,Rating,Timestamp
UserID,,,
946,20,20,19
4067,20,20,19
2529,20,20,19
340,20,20,19
5257,20,20,19


In [170]:
df13[df13["ItemID"]==25].shape

(382, 4)

In [171]:
df_ratings[df_ratings["ItemID"]==47].shape

(382, 4)

In [149]:
df11.head()

,UserID,ItemID,Negatives,Rating
0,0,25,"[1064, 174, 2791, 3373, 269, 2678, 1902, 3641,...",1
1,1,133,"[1072, 3154, 3368, 3644, 549, 1810, 937, 1514,...",1
2,2,207,"[2216, 209, 2347, 3, 1652, 3397, 383, 2905, 22...",1
3,3,208,"[3023, 1489, 1916, 1706, 1221, 1191, 2671, 81,...",1
4,4,222,"[1794, 3535, 108, 593, 466, 2048, 854, 1378, 1...",1


In [148]:
df_test_ratings.head()

,UserID,ItemID,Rating,Timestamp,Negatives
25,0,47,5,978824351,"[1976, 1881, 410, 1082, 2301, 904, 1062, 3696,..."
136,1,1737,3,978300174,"[368, 364, 2681, 1592, 3338, 1451, 2118, 74, 5..."
232,2,1900,4,978298504,"[2098, 3027, 2308, 1905, 2399, 296, 1238, 3177..."
235,3,2743,4,978294282,"[454, 750, 2167, 1585, 506, 1159, 1783, 261, 3..."
258,4,279,2,978246585,"[630, 1121, 2780, 1310, 738, 2949, 1577, 2055,..."


In [129]:
df21 = pd.read_pickle("data/movielens.test.data")
df21 = df21.sort_values(["UserID","ItemID"])
df21.shape

(6040, 5)

In [13]:
len(df2.loc[25]["Negatives"])

100

In [68]:
df2 = pd.read_pickle("data/movielens.train.data")
df2 = df2.sort_values("UserID")
df2.head()

,UserID,ItemID,Rating,Timestamp,Negatives
0,1,1193,5,978300760,"[2602, 853, 3188, 342, 3735, 3643, 2255, 274, ..."
29,1,745,3,978824268,"[2344, 2657, 2320, 1845, 180, 2007, 2478, 3683..."
30,1,2294,4,978824291,"[2608, 1636, 2381, 2374, 596, 1052, 414, 2850,..."
31,1,3186,4,978300019,"[1528, 3594, 1251, 717, 291, 3030, 1899, 3454,..."
32,1,1566,4,978824330,"[1082, 1075, 2973, 1549, 3602, 2718, 1919, 234..."


In [43]:
df3 = pd.read_csv("ncf/Data/ml-1m.train.rating", sep="\t",header=None)
df3.columns = ["UserID","ItemID","Rating","Timestamp"]
df3.head()

,UserID,ItemID,Rating,Timestamp
0,0,32,4,978824330
1,0,34,4,978824330
2,0,4,5,978824291
3,0,35,4,978824291
4,0,30,4,978824291


In [44]:
df3.to_pickle("data/movielens.train.data.original")

In [51]:
len(df3["ItemID"].unique())

3704

In [71]:
df2.ItemID.max()

3952

In [69]:
len(df2["ItemID"].unique())

3704

In [53]:
df2.head()

,UserID,ItemID,Rating,Timestamp,Negatives
25,1,48,5,978824351,"[2947, 3268, 1330, 3841, 1184, 276, 1736, 1603..."
66,2,1687,3,978300174,"[3322, 1287, 850, 2885, 3755, 1027, 540, 1985,..."
232,3,2081,4,978298504,"[2141, 1978, 1480, 3103, 915, 555, 2707, 3566,..."
243,4,1954,5,978294282,"[954, 1460, 757, 3313, 901, 3531, 2007, 1423, ..."
258,5,288,2,978246585,"[2302, 945, 2607, 3549, 881, 3372, 1511, 2986,..."


In [55]:
len(df_ratings["ItemID"].unique())

3706